## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import gc
import pretty_html_table as phtm
#import sys
# To be able to import from lib directory
#sys.path.append(r'..\lib\\')
#do not show warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
gc.collect()

69

## Datos crudos

In [3]:
#Cargar todos los datos
saber11_20171 = pd.read_csv(r'data\Saber_11__2017-1.csv')
saber11_20172 = pd.read_csv(r'data\Saber_11__2017-2.csv')
saber11_20181 = pd.read_csv(r'data\Saber_11__2018-1.csv')
saber11_20182 = pd.read_csv(r'data\Saber_11__2018-2.csv')
saber11_20191 = pd.read_csv(r'data\Saber_11__2019-1.csv')
saber11_20192 = pd.read_csv(r'data\Saber_11__2019-2.csv')

In [4]:
#Lista de DB
datos = [saber11_20171, 
         saber11_20172, 
         saber11_20181, 
         saber11_20182, 
         saber11_20191, 
         saber11_20192]

In [5]:
#Columnas a minúscula
for peri in datos:
    peri.columns = [i.lower() for i in peri.columns]

In [6]:
variables_cole = [i for i in saber11_20171.columns if 'cole' in i]
variables_estu = [i for i in saber11_20171.columns if 'estu' in i]
variables_fami = [i for i in saber11_20171.columns if 'fami' in i]

key_cole = ['cole_cod_dane_establecimiento', 
            'cole_cod_dane_sede',
            'cole_cod_mcpio_ubicacion',
            'cole_cod_depto_ubicacion',
            'cole_naturaleza',
            'cole_calendario']

key_estu = key_cole + ['fami_estratovivienda','punt_global','estu_horassemanatrabaja','periodo','estu_consecutivo']

## Colegios

In [40]:
#Concat de todos los periodos: cada fila es un colegio
key = key_cole + ['cole_nombre_sede', 'cole_mcpio_ubicacion','cole_depto_ubicacion']
colegios_2017_2019 = pd.concat([saber11_20171[key].drop_duplicates(), 
                                saber11_20172[key].drop_duplicates(),
                                saber11_20181[key].drop_duplicates(), 
                                saber11_20182[key].drop_duplicates(), 
                                saber11_20191[key].drop_duplicates(), 
                                saber11_20192[key].drop_duplicates()])

In [41]:
#Algunos duplicados por un cambio en el nombre
colegios_2017_2019.drop_duplicates(inplace=True)

In [42]:
#Revisar duplicados
colegios_2017_2019[key_cole].drop_duplicates().shape[0] == colegios_2017_2019.shape[0]

False

In [43]:
#Un solo nombre por colegio, escogido de forma alfabética
colegios_2017_2019_ = pd.pivot_table(data=colegios_2017_2019,
                            index=key_cole,
                            aggfunc={'cole_nombre_sede':'min','cole_mcpio_ubicacion':'min','cole_depto_ubicacion':'min'}).reset_index()

In [11]:
colegios_2017_2019_[key_cole].drop_duplicates().shape[0] == colegios_2017_2019_.shape[0]

True

## Estudiantes

In [176]:
#Concat de todos los periodos: cada fila es un estudiante
estudiantes_2017_2019 = pd.concat([saber11_20171[key_estu], 
                                   saber11_20172[key_estu], 
                                   saber11_20181[key_estu], 
                                   saber11_20182[key_estu], 
                                   saber11_20191[key_estu], 
                                   saber11_20192[key_estu]])

In [177]:
#Null check
estudiantes_2017_2019.isna().sum()

cole_cod_dane_establecimiento    0  
cole_cod_dane_sede               0  
cole_cod_mcpio_ubicacion         0  
cole_cod_depto_ubicacion         0  
cole_naturaleza                  0  
cole_calendario                  0  
fami_estratovivienda             26 
punt_global                      0  
estu_horassemanatrabaja          381
periodo                          0  
estu_consecutivo                 0  
dtype: int64

In [178]:
#Fill nulls
estudiantes_2017_2019['fami_estratovivienda'].fillna('AA-Sin info',inplace=True)
estudiantes_2017_2019['estu_horassemanatrabaja'].fillna('AA-Sin info',inplace=True)

In [179]:
#Juntar las dos sedes del Marymount en Atlántico
estudiantes_2017_2019['cole_cod_dane_establecimiento'] = estudiantes_2017_2019['cole_cod_dane_establecimiento'].apply(lambda x: 308001004209 if x == 308573800031 else x)
estudiantes_2017_2019['cole_cod_dane_sede'] = estudiantes_2017_2019['cole_cod_dane_sede'].apply(lambda x: 308001004209 if x == 308573800031 else x)
mask = (estudiantes_2017_2019['cole_cod_dane_sede'] == 308001004209) & (estudiantes_2017_2019['cole_cod_mcpio_ubicacion'] == 8573)
estudiantes_2017_2019['cole_cod_mcpio_ubicacion'][mask] = 8001

In [180]:
#Duplicates check
estudiantes_2017_2019[['estu_consecutivo']].drop_duplicates().shape[0] == estudiantes_2017_2019.shape[0]


True

## Cálculos

In [181]:
#Método 1: Promedio por institución sobre todos los periodos
#Cálculo del promedio de todos lo periodos por colegio y su sigma
#Promedio
cole_acumulado = pd.pivot_table(data=estudiantes_2017_2019,
                                index=key_cole,
                                aggfunc={'punt_global':'mean','estu_consecutivo':'count'}).reset_index()
cole_acumulado.rename(columns={"punt_global": "prom_punt_global","estu_consecutivo": "num_evaluados"},inplace=True)

#Desviación estándar
cole_acumulado1 = pd.pivot_table(data=estudiantes_2017_2019,
                                 index=key_cole,
                                 aggfunc={'punt_global':np.std}).reset_index() 
cole_acumulado1.rename(columns={"punt_global": "stdve_punt_global"},inplace=True)

#Resultados
cole_acumulado_final = cole_acumulado.merge(cole_acumulado1[key_cole+['stdve_punt_global']], 
                                            how='left',
                                            left_on=key_cole, 
                                            right_on=key_cole)

cole_acumulado_final['stdve_punt_global'].fillna(0, inplace=True)

#Criterio de Andes = Promedio - 1.25(Desviaciones estándar)
cole_acumulado_final['criterio_andes'] = cole_acumulado_final['prom_punt_global'] - 1.25*cole_acumulado_final['stdve_punt_global']

#True: Criterio Andes >= 310 , False: Criterio Andes < 310*
#Acá se puso en 295 porque es el valor mínimo de un colegio confirmado. Más adelante se cambia el valor con la condición correcta
cole_acumulado_final['entrada_directa'] = [False]*cole_acumulado_final.shape[0]
cole_acumulado_final['entrada_directa'][cole_acumulado_final.criterio_andes >= 295] = True


In [182]:
resultados_cole = cole_acumulado_final[(cole_acumulado_final.entrada_directa) & (cole_acumulado_final.cole_calendario == 'B')].sort_values(by='criterio_andes', ascending=False)

In [183]:
resultados_final = resultados_cole.merge(colegios_2017_2019_,
                                         left_on=key_cole, 
                                         right_on=key_cole)

In [205]:
resultados_final[resultados_final.entrada_directa=='No']

,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_mcpio_ubicacion,cole_cod_depto_ubicacion,cole_naturaleza,cole_calendario,num_evaluados,prom_punt_global,stdve_punt_global,criterio_andes,entrada_directa,cole_depto_ubicacion,cole_mcpio_ubicacion,cole_nombre_sede,confirmados_el_uniandino
58,308001004209,308001004209,8001,8,NO OFICIAL,B,209,349.11,31.67,309.52,No,ATLANTICO,BARRANQUILLA,COL MARYMOUNT -,Sin información
59,376001012606,376001012606,76001,76,NO OFICIAL,B,56,347.77,30.84,309.22,No,VALLE,CALI,ASPAEN LICEO TACURÍ,Sin información
60,311769003474,311769003474,11001,11,NO OFICIAL,B,352,344.81,28.61,309.05,No,BOGOTA,BOGOTÁ D.C.,COL ANDINO,Entrada directa
61,311001047863,311001047863,25126,25,NO OFICIAL,B,127,343.64,27.85,308.82,No,CUNDINAMARCA,CAJICA,COL FUND CARDENAL JHON HENRY NEWMAN,Sin información
62,325175000549,325175000549,25175,25,NO OFICIAL,B,164,342.60,28.10,307.47,No,CUNDINAMARCA,CHIA,GIMN BRITANICO,Sin información
63,311848000936,311848000936,11001,11,NO OFICIAL,B,244,343.97,29.27,307.38,No,BOGOTA,BOGOTÁ D.C.,COL SANTA MARIA,Entrada directa
64,308001077745,308001077745,8001,8,NO OFICIAL,B,149,349.46,33.67,307.36,No,ATLANTICO,BARRANQUILLA,COL REAL (ROYAL SCHOOL) -,Sin información
65,463190000443,463190000443,63690,63,NO OFICIAL,B,35,348.46,33.13,307.04,No,QUINDIO,SALENTO,GIMN INGLES,Sin información
66,405607000476,405607000476,5607,5,NO OFICIAL,B,87,347.37,32.59,306.63,No,ANTIOQUIA,RETIRO,COL GIMN VERMONT,Sin información
67,311001107513,311001107513,11001,11,NO OFICIAL,B,90,340.92,27.85,306.12,No,BOGOTA,BOGOTÁ D.C.,COL VICTORIA -,Sin información


In [185]:
resultados_final[resultados_final['cole_nombre_sede'].str.contains("CORALES")]

,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_mcpio_ubicacion,cole_cod_depto_ubicacion,cole_naturaleza,cole_calendario,num_evaluados,prom_punt_global,stdve_punt_global,criterio_andes,entrada_directa,cole_depto_ubicacion,cole_mcpio_ubicacion,cole_nombre_sede
79,308001012805,308001012805,8573,8,NO OFICIAL,B,17,341.529412,30.590271,303.291572,True,ATLANTICO,PUERTO COLOMBIA,ASPAEN GIMNASIO LOS CORALES


In [186]:
#Confirmados por El Uniandino 
confirmados_el_uniandino = [311769001781, 
                            311848001011, 
                            425758800009,
                            311769003474,
                            311001105430,
                            311769000866,
                            368001000516,
                            376001026968,
                            311848000341,
                            311001087288,
                            311848000278,
                            425377800011,
                            311769000921,
                            311769003342,
                            311001019941,
                            311001045160,
                            311848002424,
                            311769000165,
                            311848000936,
                            425377000529,
                            368276000826,
                            319001002895,
                            308001073952,
                            311848000812,
                            311848002262,
                            311769001846,
                            311848000286,
                            311769000475,
                            376001002279,
                            311769003938,
                            311769004292,
                            466001005641,
                            311001019568,
                            311001010820,
                            311769002818]

nombres_confirmados_el_uniandino = ['COL ABRAHAM LINCOLN',
                                    'COL ANGLO COLOMBIANO',
                                    'COL CAMPOALEGRE LTDA - SEDE PRINCIPAL',
                                    'COL ANDINO',
                                    'COL LA COLINA',
                                    'FUND NUEVO MARYMOUNT',
                                    'COL NUEVO CAMBRIDGE',
                                    'COLEGIO FREINET',
                                    'GIMN CAMPESTRE',
                                    'GIMN LA MONTAÑA',
                                    'GIMN VERMONT',
                                    'COL. HACIENDA LOS ALCAPARROS',
                                    'COL HELVETIA',
                                    'COL LOS NOGALES',
                                    'COL NUEVA GRANADA',
                                    'FUND GIMN LOS PORTALES',
                                    'COL SAN CARLOS',
                                    'COL SAN JORGE DE INGLATERRA',
                                    'COL SANTA MARIA',
                                    'COL TILATA - SEDE PRINCIPAL',
                                    'COL LA QUINTA DEL PUENTE',
                                    'COL GIMNASIO CALIBIO',
                                    'COLEGIO ALTAMIRA',
                                    'GIMN FEMENINO',
                                    'COL ITALIANO LEONARDO DA VINCI',
                                    'COL SAN TARSICIO',
                                    'GIMN DE LOS CERROS',
                                    'ASPAEN GIMN IRAGUA',
                                    'COL COLOMBO BRITÁNICO',
                                    'COL BILING BUCKINGHAM',
                                    'COL SAN MATEO APOSTOL',
                                    'LIC CAMPESTRE DE PEREIRA',
                                    'LIC FRANCES LOUIS PASTEUR',
                                    'COL SAN BARTOLOME LA MERCED',
                                    'COL. INTERNACIONAL DE EDUCACIÓN INTEGRAL - CIEDI LTDA - SEDE PRINCIPAL']

In [187]:
len(confirmados_el_uniandino)

35

In [188]:
#Pretty results
#Nombres
resultados_final['cole_nombre_sede'] = [i.replace('COLEGIO','COL') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('COL.','COL') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('SEDE','') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('PRINCIPAL','') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('PRINCIPAL','') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('GIMNASIO','GIMN') for i in resultados_final['cole_nombre_sede']]
resultados_final['cole_nombre_sede'] = [i.replace('GIMN.','GIMN') for i in resultados_final['cole_nombre_sede']]

#Números
resultados_final['prom_punt_global'] = [round(i,2) for i in resultados_final['prom_punt_global']]
resultados_final['stdve_punt_global'] = [round(i,2) for i in resultados_final['stdve_punt_global']]
resultados_final['criterio_andes'] = [round(i,2) for i in resultados_final['criterio_andes']]

#Booleanos
resultados_final['confirmados_el_uniandino'] = ['Sin información']*resultados_final.shape[0]
resultados_final['confirmados_el_uniandino'][resultados_final.cole_cod_dane_establecimiento.isin(confirmados_el_uniandino)] = 'Entrada directa'
resultados_final['entrada_directa'][resultados_final.entrada_directa] = 'Sí'
#Condición correcta
resultados_final['entrada_directa'][resultados_final.criterio_andes<310] = 'No'

In [189]:
pd.set_option('display.max_colwidth', 0)
export = ['cole_nombre_sede','cole_depto_ubicacion','prom_punt_global','stdve_punt_global','entrada_directa']

In [190]:
table_pretty = resultados_final[export+['confirmados_el_uniandino']]

table_pretty.rename(columns={"cole_nombre_sede": "Colegio", 
                             "cole_depto_ubicacion": "Departamento",
                             "prom_punt_global": "Promedio Saber 11",
                             "stdve_punt_global": "$\sigma$ Saber 11",
                             "criterio_andes":"Criterio de Los Andes",
                             "entrada_directa":"¿Entra según criterio?",
                             "confirmados_el_uniandino":"Confirmado por El Uniandino"},inplace=True)

table_pretty.index += 1 

In [191]:
len(confirmados_el_uniandino) == len(nombres_confirmados_el_uniandino)

True

In [ ]:
#Html table
<p><table border="0" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">Colegio</th>
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">Departamento</th>
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">Promedio Saber 11</th>
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">Sigma Saber 11</th>
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">¿Entra según criterio?</th>
      <th style = "background-color: #FFFFFF;font-family: Arial;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px">Confirmado por El Uniandino</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LA QUINTA DEL PUENTE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">SANTANDER</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">394.46</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.03</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL NUEVO CAMBRIDGE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">SANTANDER</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">395.42</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.15</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BILINGUE DIANA OESE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">393.16</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.66</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL PHILADELPHIA INTERNACIONAL -</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">373.85</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">22.03</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN JORGE DE INGLATERRA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">375.27</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">24.16</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LOS NOGALES</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">371.65</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">23.38</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN CARLOS</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">374.22</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.07</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN VERMONT</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">370.35</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">24.68</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SANTA FRANCISCA ROMANA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">368.39</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">24.02</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL CAMPOALEGRE LTDA -</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">365.04</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">21.79</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LA COLINA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">372.46</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.5</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN MATEO APOSTOL</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">366.99</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.16</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BILING BUCKINGHAM</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">361.92</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">22.14</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL  MONTESSORI BRITISH SCHOOL</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">366.9</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.88</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ABRAHAM LINCOLN</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">368.01</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.93</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN ALESSANDRO VOLTA -</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">362.89</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.53</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL FREINET</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">363.32</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.76</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LEONARDO DA VINCI</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ANTIOQUIA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">361.71</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.72</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL GIMN LA COLINA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">354.41</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">21.89</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL HISPANOAMERICANO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">361.03</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.54</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">LIC CAMPESTRE DE PEREIRA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">RISARALDA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">357.06</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">24.6</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN LA MONTAÑA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">364.34</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.22</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL MONTESSORI</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ANTIOQUIA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">357.11</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.05</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN DE LOS CERROS</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">361.14</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.56</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">FUND NUEVO MARYMOUNT</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">309.52</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">25.61</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN BARTOLOME LA MERCED</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">355.07</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.22</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN CAMPESTRE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">356.33</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.41</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">FUND GIMN LOS PORTALES</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">355.66</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.19</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL CALATRAVA -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">360.37</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.05</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL INTERNACIONAL LA SIERRA -</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CESAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">348.92</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">22.23</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN TARSICIO</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">355.7</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.1</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL HELVETIA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">352.8</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">25.94</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN CARTAGENA-ASPAEN</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOLIVAR</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">362.88</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">34.16</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BILING CLERMONT -</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">352.31</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">25.77</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CORP. EDUC. GIMN ALTAIR DE C/GENA.</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOLIVAR</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">355.16</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.06</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL JOSE MAX LEON</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">352.43</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">25.95</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ITALIANO LEONARDO DA VINCI</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">354.64</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.95</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL GIMN CALIBIO</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CAUCA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">354.2</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.27</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL INTERNACIONAL DE EDUCACIÓN INTEGRAL - CIEDI LTDA -</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">353.02</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.39</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL PANAMERICANO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">SANTANDER</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">356.16</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.95</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ALTAMIRA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">354.84</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.5</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASPAEN GIMN CARTAGENA DE INDIAS</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOLIVAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">356.71</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.59</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LIC FRANCES DE PEREIRA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">RISARALDA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">354.12</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.93</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN CAMP SAN RAFAEL</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">350.94</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.86</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL HACIENDA LOS ALCAPARROS</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">350.43</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.95</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">LIC FRANCES LOUIS PASTEUR</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">351.56</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.98</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL TILATA -</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">350.94</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.39</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN INGLES</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">QUINDIO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">352.83</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.03</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BRITANICO DE CARTAGENA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOLIVAR</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">357.33</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">35.79</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL COLOMBO BRITÁNICO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">352.3</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.96</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL HEBREO UNION -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">348.63</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.33</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BERCHMANS</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">353.23</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.05</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASPAEN GIMN IRAGUA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">349.23</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.93</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN FEMENINO</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">346.04</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.47</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CORPÓRACION GIMN ALEMÁN FRIENDRICH VON SCHILLER -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">337.25</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">21.06</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CORPORACIÓN CULTURAL COL ALEMÁN -  PRICIPAL</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">349.83</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.24</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #E1FFE3;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL EL CAMINO ACADEMY</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">349.27</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.97</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL CUMBRES</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">348.2</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.44</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sí</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL MARYMOUNT</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.77</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.67</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASPAEN LICEO TACURÍ</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.77</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.84</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ANDINO</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">344.81</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.61</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL FUND CARDENAL JHON HENRY NEWMAN</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">343.64</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.85</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN BRITANICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">342.6</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.1</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SANTA MARIA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">343.97</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.27</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL REAL (ROYAL SCHOOL) -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">349.46</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.67</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN INGLES</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">QUINDIO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">348.46</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.13</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL GIMN VERMONT</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ANTIOQUIA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.37</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">32.59</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL VICTORIA -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">340.92</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">27.85</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ANGLO COLOMBIANO</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.03</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.45</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL JORGE WASHINGTON -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOLIVAR</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.9</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">34.22</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL SAN JOSÉ -</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">337.75</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">26.21</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BILING RICHMOND</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">343.01</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.71</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BOLIVAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">346.72</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.87</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ANGLO AMERICANO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">340.97</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.3</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CORPORACION COL CUMBRES</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ANTIOQUIA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">344.78</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">32.39</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL DE INGLATERRA (THE ENGLISH SCHOOL) -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.21</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.6</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL CAMPESTRE AMERICANO -</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CAUCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">332.21</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">22.51</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL MAYOR DE LOS ANDES -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.53</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.19</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL INTERNACIONAL LOS CAÑAVERALES</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">338.68</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">28.09</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASPAEN GIMN LOS CORALES</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.53</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.59</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL FUND.   BILING . DE VALLEDUPAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CESAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">348.53</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">36.4</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN LOS CAOBOS</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">339.93</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">30.21</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CENT EDUC Y CULT ESPAÑOL REYES CATOLICOS</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.81</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">32.49</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LA MEDALLA  MILAGROSA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">337.48</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.06</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ROCHESTER</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">338.47</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.91</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL FUNDACION LIC INGLES</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">RISARALDA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">346.79</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">36.87</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">LICEO BENALCAZAR</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">343.98</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">34.71</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LOS ANDES</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CAUCA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.61</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">37.67</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL INTERNACIONAL SEK COLOMBIA - TRINIDAD DEL  MONTE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CUNDINAMARCA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">339.43</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.13</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL LA ARBOLEDA -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">347.4</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">37.91</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN FONTANA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">339.9</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">32.09</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">GIMN DEL NORTE -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">338.66</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">31.15</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">LICEO CAMPESTRE INGLES DEL SUR -</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">325.0</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">20.3</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASPAEN COL JUANAMBÚ -</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">346.25</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">37.45</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL BRITANICO INTERNACIONAL</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ATLANTICO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.08</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.32</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL JEFFERSON</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">VALLE</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">342.71</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">35.25</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL COLOMBO HEBREO</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">334.24</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">29.14</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">ASOCIACION COL GRANADINO</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">CALDAS</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">342.08</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">35.44</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL COLOMBO GALES</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">338.77</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">33.77</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FFFFFF;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL ALBANIA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">LA GUAJIRA</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.89</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">37.38</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #D9E1F2;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Sin información</td>
    </tr>
    <tr>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">COL NUEVA GRANADA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">BOGOTA</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">341.81</td>
      <td style = "background-color: #FFDAB9;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">37.41</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">No</td>
      <td style = "background-color: #FF8578;font-family: Arial;text-align: left;padding: 0px 0px 0px 0px">Entrada directa</td>
    </tr>
  </tbody>
</table></p>

In [ ]:
#Método 2: Promedio por periodo-institución y luego por institución
#Cálculo del promedio de cada periodo por colegio, luego sacar el promedio por colegio y su sigma
#Promedio periodo-colegio
cole_acumulado_periodo = pd.pivot_table(data=estudiantes_2017_2019,
                                index=key_cole + ['periodo'],
                                aggfunc={'punt_global':'mean','estu_consecutivo':'count'}).reset_index()
cole_acumulado_periodo.rename(columns={"punt_global": "prom_punt_global","estu_consecutivo": "num_evaluados"},inplace=True)

#Promedio colegio
cole_acumulado_2 = pd.pivot_table(data=cole_acumulado_periodo,
                                index=key_cole,
                                aggfunc={'prom_punt_global':'mean','num_evaluados':'sum'}).reset_index()


#Desviación estándar
cole_acumulado_2_1 = pd.pivot_table(data=cole_acumulado_periodo,
                                 index=key_cole,
                                 aggfunc={'prom_punt_global':np.std}).reset_index() 
cole_acumulado_2_1.rename(columns={"prom_punt_global": "stdve_punt_global"},inplace=True)

#Resultados
cole_acumulado_final_1 = cole_acumulado_2.merge(cole_acumulado_2_1[key_cole+['stdve_punt_global']], 
                                                how='left',
                                                left_on=key_cole, 
                                                right_on=key_cole)

cole_acumulado_final_1.fillna(0, inplace=True)


#Criterio de Andes = Promedio - 1.25(Desviaciones estándar)
cole_acumulado_final_1['criterio_andes'] = cole_acumulado_final_1['prom_punt_global'] - 1.25*cole_acumulado_final_1['stdve_punt_global']

#True: Criterio Andes >= 310 , False: Criterio Andes < 310
cole_acumulado_final_1['entrada_directa'] = [False]*cole_acumulado_final_1.shape[0]
cole_acumulado_final_1['entrada_directa'][cole_acumulado_final_1.criterio_andes >= 310] = True


In [ ]:
resultados_cole_1 = cole_acumulado_final_1[(cole_acumulado_final_1.entrada_directa) & (cole_acumulado_final_1.cole_calendario == 'B')].sort_values(by='criterio_andes', ascending=False)

In [ ]:
resultados_final_1 = resultados_cole_1.merge(colegios_2017_2019_,
                                             left_on=key_cole, 
                                             right_on=key_cole)

In [ ]:
coles = ['cole_depto_ubicacion', 
         'cole_mcpio_ubicacion', 
         'cole_nombre_sede',
         'cole_calendario',
         'num_evaluados',
         'prom_punt_global',
         'stdve_punt_global',
         'criterio_andes',
         'entrada_directa']
resultados_final[coles].to_csv(r'data\59_colegios.csv')

## Insights

In [192]:
#Saber11_Histórico_vs_Estrato
estudiantes_2017_2019['fami_estratovivienda'][estudiantes_2017_2019['fami_estratovivienda']=='-'] = ['AA-Sin info']*estudiantes_2017_2019.shape[0]
estudiantes_2017_2019['fami_estratovivienda'][estudiantes_2017_2019['fami_estratovivienda']=='Sin Estrato'] = ['A-Sin Estrato']*estudiantes_2017_2019.shape[0]
data = estudiantes_2017_2019.sort_values(by="fami_estratovivienda")

data['estrato'] = data['fami_estratovivienda']
distribucion = pd.pivot_table(data=data,index=['fami_estratovivienda'],aggfunc={'estrato':'count'}).reset_index()

fig = plt.figure(figsize=(11,12))
gs = fig.add_gridspec(4, 4)
ax1 = plt.subplot(gs[0:3, :])
ax3 = plt.subplot(gs[3, :])
ax2 = ax1.twinx()

sns.boxplot(x="fami_estratovivienda", 
            y="punt_global", 
            data=data,
            color='lightblue',
            ax=ax1)

ax1.set_title('Saber 11 vs. Estrato - Histórico Nacional 2017-2019',y=1,fontsize=15)
ax1.set_xlabel('',  fontsize=14)
ax1.set_ylabel('Promedio del puntaje global en el Saber 11', fontsize=14)
ax1.set_ylim(-10,500)
ax1.tick_params(axis='both', which='major', labelsize=13)

ax2.plot([310]*8,
         color='r',
         ls='--', 
         linewidth=2, 
         label='Corte mínimo de Los Andes (310)')
ax2.set_ylim(-10,500)
ax2.legend(loc="best",fontsize=15)
ax2.tick_params(axis='both', which='major', labelsize=13)

ax3.bar(x=distribucion['fami_estratovivienda'], 
        height=distribucion['estrato']/sum(distribucion['estrato']),
        color='mistyrose')
ax3.set_xlabel('Estrato de la familia del estudiante',  fontsize=14,y=15)
ax3.set_ylabel('%Población evaluada',  fontsize=14)
fig.tight_layout()


plt.savefig(r'C:\Users\Usuario\Pictures\Saber11\saber11_estrato_H.png')
plt.close()

In [194]:
#Saber11_B_vs_Estrato
data = estudiantes_2017_2019[estudiantes_2017_2019.periodo.isin([20191,20181,20171])]
data['fami_estratovivienda'][data['fami_estratovivienda']=='-'] = ['AA-Sin info']*data.shape[0]
data['fami_estratovivienda'][data['fami_estratovivienda']=='Sin Estrato'] = ['A-Sin Estrato']*data.shape[0]
data = data.sort_values(by="fami_estratovivienda")

data['estrato'] = data['fami_estratovivienda']
distribucion = pd.pivot_table(data=data,index=['fami_estratovivienda'],aggfunc={'estrato':'count'}).reset_index()

fig = plt.figure(figsize=(11,12))
gs = fig.add_gridspec(4, 4)
ax1 = plt.subplot(gs[0:3, :])
ax3 = plt.subplot(gs[3, :])
ax2 = ax1.twinx()

sns.boxplot(x="fami_estratovivienda", 
            y="punt_global", 
            data=data,
            color='lightblue',
            ax=ax1)

ax1.set_title('Saber 11 vs. Estrato - Histórico calendario B 2017-2019',y=1,fontsize=15)
ax1.set_xlabel('',  fontsize=14)
ax1.set_ylabel('Promedio del puntaje global en el Saber 11', fontsize=14)
ax1.set_ylim(-10,500)
ax1.tick_params(axis='both', which='major', labelsize=13)

ax2.plot([310]*8,
         color='r',
         ls='--', 
         linewidth=2, 
         label='Corte mínimo de Los Andes (310)')
ax2.set_ylim(-10,500)
ax2.legend(loc="best",fontsize=15)
ax2.tick_params(axis='both', which='major', labelsize=13)

ax3.bar(x=distribucion['fami_estratovivienda'], 
        height=distribucion['estrato']/sum(distribucion['estrato']),
        color='mistyrose')
ax3.set_xlabel('Estrato de la familia del estudiante',  fontsize=14,y=15)
ax3.set_ylabel('%Población evaluada',  fontsize=14)
fig.tight_layout()


plt.savefig(r'C:\Users\Usuario\Pictures\Saber11\saber11_estrato_B.png')
plt.close()

In [195]:
#Estrato_Confirmados
top_59 = resultados_final[resultados_final.criterio_andes>=310].cole_cod_dane_establecimiento.drop_duplicates().values
data = estudiantes_2017_2019[(estudiantes_2017_2019.cole_cod_dane_establecimiento.isin(confirmados_el_uniandino+list(top_59)))].sort_values(by='fami_estratovivienda')
#data = data[data.cole_cod_dane_establecimiento.isin([top_59])]
data['estrato'] = data['fami_estratovivienda']
distribucion = pd.pivot_table(data=data,index=['fami_estratovivienda'],aggfunc={'estrato':'count'}).reset_index()

data_all = estudiantes_2017_2019.sort_values(by='fami_estratovivienda')
data_all['estrato'] = data_all['fami_estratovivienda']
distribucion_all = pd.pivot_table(data=data_all,index=['fami_estratovivienda'],aggfunc={'estrato':'count'}).reset_index()


fig = plt.figure(figsize=(11,6))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)

ax1.set_title('Estrato de los colegios con admisión automática según criterio + confirmados', y=1, fontsize=15)
ax1.bar(x=distribucion['fami_estratovivienda'], 
        height=distribucion['estrato']/sum(distribucion['estrato']),
        color='mistyrose')
ax1.set_ylabel('%Población',  fontsize=14)
ax1.set_ylim(0,0.5)

ax2.set_title('Estrato estudiantes de los colegios a nivel nacional',y=1,fontsize=15)
ax2.bar(x=distribucion_all['fami_estratovivienda'], 
        height=distribucion_all['estrato']/sum(distribucion_all['estrato']),
        color='mistyrose')
ax2.set_xlabel('Estrato de la familia del estudiante',  fontsize=14,y=15)
ax2.set_ylabel('%Población',  fontsize=14)
ax2.set_ylim(0,0.5)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
plt.savefig(r'C:\Users\Usuario\Pictures\Saber11\estratp_auto.png')
plt.close()

In [196]:
len(top_59)

58

In [197]:
#Regiones_Confirmados
data = estudiantes_2017_2019[estudiantes_2017_2019.cole_cod_dane_establecimiento.isin(confirmados_el_uniandino+list(top_59))]
data = data.merge(colegios_2017_2019_,
                  how='left',
                  left_on=key_cole, 
                  right_on=key_cole).sort_values(by='cole_depto_ubicacion')

data['depto'] = data['cole_depto_ubicacion']
distribucion = pd.pivot_table(data=data,index=['cole_depto_ubicacion'],aggfunc={'depto':'count'}).reset_index()

data_all = estudiantes_2017_2019
data_all = data_all.merge(colegios_2017_2019_,
                          how='left',
                          left_on=key_cole, 
                          right_on=key_cole).sort_values(by='cole_depto_ubicacion')

data_all['depto'] = data_all['cole_depto_ubicacion']
distribucion_all = pd.pivot_table(data=data_all,index=['cole_depto_ubicacion'],aggfunc={'depto':'count'}).reset_index()

fig = plt.figure(figsize=(11,6))
ax1 = plt.subplot(211)
ax1.set_title('Departamento de los colegios con admisión automática según criterio + confirmados',y=1,fontsize=15)
ax1.bar(x=distribucion['cole_depto_ubicacion'], 
        height=distribucion['depto']/sum(distribucion['depto']),
        color='mistyrose')
ax1.set_ylabel('%Población',  fontsize=14)
ax1.set_ylim(0,1)
ax1.tick_params(axis='x', which='major', labelsize=9,rotation=25)


ax2 = plt.subplot(212)
ax2.set_title('Departamento de los colegios a nivel nacional', y=1, fontsize=15)
ax2.bar(x=distribucion_all['cole_depto_ubicacion'], 
        height=distribucion_all['depto']/sum(distribucion_all['depto']),
        color='mistyrose')
ax2.set_xlabel('Departamento del colegio del estudiante',  fontsize=14,y=15)
ax2.set_ylabel('%Población',  fontsize=14)
ax2.tick_params(axis='x', which='major', labelsize=9,rotation=90)
ax2.set_ylim(0,0.3)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.6)
plt.savefig(r'C:\Users\Usuario\Pictures\Saber11\depto_auto.png')
plt.close()